In [68]:
import pandas as pd
import tkinter as tk
import warnings

warnings.filterwarnings('ignore')

In [69]:
app = tk.Tk()
app.title('RECOMMENDER SYSTEM')
app.geometry('500x200')
 
tk.Label(app, text='Movie Watched:').place(x=25,y=25)
tk.Label(app, text='Recommended ').place(x=25,y=100)
tk.Label(app, text='also try').place(x=25,y=125)

movie_var = tk.Variable(app)
tk.Entry(app, textvariable=movie_var, width=28).place(x=150,y=25)

recommend_var = tk.Variable(app)
recommend_var.set('Type movie name..')
tk.Label(app, textvariable=recommend_var).place(x=150,y=100)

also_try_var = tk.Variable(app)
also_try_var.set('Type movie name..')
tk.Label(app, textvariable=also_try_var).place(x=150,y=125)
def find_recommendation():
    pass
tk.Button(app, text='Find Suggestion').place(x=125,y=60)

app.mainloop()

In [70]:
cols='user_id item_id rating timestamp'.split()
df = pd.read_csv('u.data',sep='\t',names=cols)
movie_cols = ['item_id','title']+[str(i) for i in range(22)] 
movie_titles = pd.read_csv('u.itemmkg',sep='|', names = movie_cols)
movie_titles = movie_titles[['item_id','title']]
data = pd.merge(df,movie_titles, on='item_id')
rating = pd.DataFrame(data.groupby('title')['rating'].mean())
rating['count']=data['title'].value_counts()
pivot_df = data.pivot_table(index = 'user_id', columns = 'title', values = 'rating')

app = tk.Tk()
app.title('RECOMMENDER SYSTEM')
app.geometry('500x200')


tk.Label(app, text='Movie Watched:').place(x=25,y=25)
tk.Label(app, text='Recommended ').place(x=25,y=100)
tk.Label(app, text='also try').place(x=25,y=125)

movie_var = tk.Variable(app)
tk.Entry(app, textvariable=movie_var, width=28).place(x=150,y=25)

recommend_var = tk.Variable(app)
recommend_var.set('Type movie name..')
tk.Label(app, textvariable=recommend_var).place(x=150,y=100)

also_try_var = tk.Variable(app)
also_try_var.set('Type movie name..')
tk.Label(app, textvariable=also_try_var).place(x=150,y=125)

def find_recommendation():
    movie = movie_var.get().lower().strip()
    try:
        movie = movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0]
        movie_var.set(movie)
    except IndexError:
        recommend_var.set('Movie Not Found')
        also_try_var.set('Movie Not Found')
    else:
        corr_df = pd.DataFrame(pivot_df.corrwith(pivot_df[movie]), columns = ['correlation'])
        corr_df.dropna(inplace=True)
        corr_df = corr_df.join(rating['count'])
        recommend_var.set(corr_df[(corr_df['count']>200)&(corr_df['correlation']>0.4)].sort_values(by='correlation',ascending=False).index[1])
        also_try_var.set(corr_df[corr_df['correlation']>0.4].sort_values(by = 'correlation', ascending=False).index[0])
        
    

    
tk.Button(app, text='Find Suggestion',command=find_recommendation, bg='#ee0033', fg='#ffffff').place(x=125,y=60)

app.mainloop()

In [74]:
movie = 'star wars'

In [75]:
movie_titles['title'].apply(lambda x: movie in x.lower())

0       False
1       False
2       False
3       False
4       False
        ...  
1677    False
1678    False
1679    False
1680    False
1681    False
Name: title, Length: 1682, dtype: bool

In [76]:
movie_titles[movie_titles['title'].apply(lambda x: movie in x.lower())]  #logical indexing er get the line for which it is true

,item_id,title
49,50,Star Wars (1977)


In [77]:
movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0] #by using.values we made an numpy array of above line given in output then we apply indexing to get the name

'Star Wars (1977)'

In [72]:
movie_titles['title'][movie_titles['title'].apply(lambda x: 'terminator' in x.lower())]  # this will give two suggestions therefore we will apply .values(0) to let them know that we want first one

95     Terminator 2: Judgment Day (1991)
194               Terminator, The (1984)
Name: title, dtype: object

In [78]:
movie_titles['title'][movie_titles['title'].apply(lambda x: 'terminator' in x.lower())].values[0]

'Terminator 2: Judgment Day (1991)'